# Phase 2: Find Processing Techniques

In [14]:
%reload_ext autoreload
%autoreload 2

In [2]:
# connects to utils and run a test for connectivity
from Utils.test_class_func import Test_py 
from Utils.test_class_func import test_py  
print(Test_py("My_Test_Class").print_(), "\n")
print(test_py("My_Test_Function"))

Class -> Try if python utils connects to notebook: My_Test_Class 

Function -> Try if python utils connects to notebook: My_Test_Function


In [2]:
# import functions 
from Utils.preporcessing_utils import data_loading 
from Utils.preporcessing_utils import labels_encoding
from Utils.preporcessing_utils import split_data
# from Utils.preporcessing_utils import image_iterators
from Utils.preporcessing_utils import ablation
# from Utils.models_utils import Basic_Custom_CNN
# from Utils.evaluation_utils import Evaluation
from Utils.visualizations_utils import Visualization
# from Utils.save_data_utils import Save_Data
from Utils.run_model_utils import run_model

2025-09-03 08:59:29.037650: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-03 08:59:30.016390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756904370.161516   31599 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756904370.371317   31599 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-03 08:59:31.433027: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
# import libraries
import json 

## Data Preparation and Basic Preprocessing

In [4]:
# Loads data
train_df, test_df = data_loading("train_full.csv", "test_full.csv")

In [5]:
# Create dataframe and transform(encodes) pathology labels
train_df, test_df = labels_encoding(train_df, test_df)

In [6]:
train_df.columns

Index(['image_id', 'image_type', 'image_path', 'series_uid', 'subject_id',
       'study_uid', 'breast_density', 'breast_side', 'image_view',
       'abnormality_type', 'pathology', 'split', 'label'],
      dtype='object')

In [7]:
# Split data
train_data, val_data, test_data = split_data(train_df, test_df, 0.11)

Train set: 1889 cases, 70.35 %
Validation set: 234 cases, 8.72 %
Test set: 562 cases, 20.93 %


## Create Preporcessing Techniques Groups: Ablation

In [8]:
# create groups using ablation
options = ['apply_background_removal',
           'apply_crop',
           'apply_noise_reduction',
           'apply_contrast_enhancement',
           'apply_edge_enhancement',
           'apply_lbp_texturizer']

y_true = test_data["label"]

In [9]:
# create group of techniques to try
techniques_groups = ablation(options)

In [10]:
techniques_groups["Baseline Basic Preporcessing"]

{'apply_background_removal': False,
 'apply_crop': False,
 'apply_noise_reduction': False,
 'apply_contrast_enhancement': False,
 'apply_edge_enhancement': False,
 'apply_lbp_texturizer': False}

In [11]:
baseline_techniques = {"Baseline Basic Preporcessing": techniques_groups["Baseline Basic Preporcessing"]}

In [12]:
# train_data = train_data[:20]
# val_data = val_data[:10]
# test_data = test_data[:10]
# y_true = y_true[:10]

## Check Basic Models Performance

### Baseline model with ImageGenerator
This models uses the custom CNN with 10 epochs and the tf.ImageDataGenerator image iterator. <br><br> 
The following model call will not run because it is not connected the image iterator funtion anymore becaue the function was updated. <br> 
This output was kept for reference only. 
If needed, please see **run_model_utils.py** for running instructions.  

In [12]:
# runs baseline model with an image dataset generator 
run_model((train_data, val_data, test_data), techniques_groups, epochs=10, change="Baseline(tf.ImageDataGenerator)")

Training Custom10 - Baseline Basic Preporcessing - Baseline(tf.ImageDataGenerator)
Found 1889 validated image filenames.
Found 234 validated image filenames.
Found 562 validated image filenames.


I0000 00:00:1756665577.419168   33308 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1756665588.778667   33462 service.cc:148] XLA service 0x739a240099a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756665588.778734   33462 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-08-31 14:39:48.797362: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1756665588.887976   33462 cuda_dnn.cc:529] Loaded cuDNN version 90501
E0000 00:00:1756665591.911711   33462 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756665592.022091   33462 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


 1/60 ━━━━━━━━━━━━━━━━━━━━ 9:03 9s/step - accuracy: 0.5000 - loss: 0.6927

...

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 297s 5s/step - accuracy: 0.5272 - loss: 0.6929 - val_accuracy: 0.5171 - val_loss: 0.6927
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 280s 5s/step - accuracy: 0.5092 - loss: 0.6932 - val_accuracy: 0.5171 - val_loss: 0.6931
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 280s 5s/step - accuracy: 0.5023 - loss: 0.6931 - val_accuracy: 0.5171 - val_loss: 0.6931
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 280s 5s/step - accuracy: 0.5022 - loss: 0.6930 - val_accuracy: 0.5171 - val_loss: 0.6926
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 281s 5s/step - accuracy: 0.5224 - loss: 0.6927 - val_accuracy: 0.5171 - val_loss: 0.6930
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 290s 5s/step - accuracy: 0.5211 - loss: 0.6927 - val_accuracy: 0.5171 - val_loss: 0.6930
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 292s 5s/step - accuracy: 0.5177 - loss: 0.6930 - val_accuracy: 0.5171 - val_loss: 0.6933
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 281s 5s/step - accuracy: 0.5164 - loss: 0.6926 - val_accuracy: 0.5171 - v

/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 322s 5s/step - accuracy: 0.4842 - loss: 0.6932 - val_accuracy: 0.5171 - val_loss: 0.6930
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 344s 6s/step - accuracy: 0.4977 - loss: 0.6932 - val_accuracy: 0.5171 - val_loss: 0.6930
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 336s 6s/step - accuracy: 0.5133 - loss: 0.6929 - val_accuracy: 0.5171 - val_loss: 0.6928
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 319s 5s/step - accuracy: 0.5165 - loss: 0.6927 - val_accuracy: 0.5171 - val_loss: 0.6930
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 302s 5s/step - accuracy: 0.5216 - loss: 0.6929 - val_accuracy: 0.5171 - val_loss: 0.6926
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 311s 5s/step - accuracy: 0.5262 - loss: 0.6921 - val_accuracy: 0.5171 - val_loss: 0.6929
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 293s 5s/step - accuracy: 0.4927 - loss: 0.6934 - val_accuracy: 0.5171 - val_loss: 0.6929
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 309s 5s/step - accuracy: 0.5159 - loss: 0.6928 - val_accuracy: 0.5171 - v

/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 320s 5s/step - accuracy: 0.4873 - loss: 0.6932 - val_accuracy: 0.5171 - val_loss: 0.6931
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 310s 5s/step - accuracy: 0.4752 - loss: 0.6932 - val_accuracy: 0.5171 - val_loss: 0.6929
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 322s 5s/step - accuracy: 0.5305 - loss: 0.6925 - val_accuracy: 0.5171 - val_loss: 0.6927
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 314s 5s/step - accuracy: 0.5188 - loss: 0.6927 - val_accuracy: 0.5171 - val_loss: 0.6928
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 308s 5s/step - accuracy: 0.5194 - loss: 0.6927 - val_accuracy: 0.5171 - val_loss: 0.6928
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 313s 5s/step - accuracy: 0.5261 - loss: 0.6924 - val_accuracy: 0.5171 - val_loss: 0.6926
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 304s 5s/step - accuracy: 0.5055 - loss: 0.6933 - val_accuracy: 0.5171 - val_loss: 0.6927
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 317s 5s/step - accuracy: 0.4969 - loss: 0.6936 - val_accuracy: 0.5171 - v

### Baseline model with Image tf.Dataset Generator
This models uses the custom CNN with 10 epochs and the tf.Dataset image iterator  

In [14]:
# runs baseline model with an image dataset generator 
run_model((train_data, val_data, test_data), techniques_groups, epochs=10, change="Baseline(tf.Dataset)")

Training Custom10 - Baseline Basic Preporcessing - Baseline(tf.Dataset)
Epoch 1/10


2025-09-03 10:39:11.765416: I tensorflow/core/framework/local_rendezvous.cc:428] Local rendezvous send item cancelled. Key hash: 15106503898888123074
2025-09-03 10:39:11.765632: I tensorflow/core/framework/local_rendezvous.cc:428] Local rendezvous send item cancelled. Key hash: 15106503898888123074
2025-09-03 10:39:11.765637: I tensorflow/core/framework/local_rendezvous.cc:428] Local rendezvous send item cancelled. Key hash: 11736988844813115236
2025-09-03 10:39:11.765639: I tensorflow/core/framework/local_rendezvous.cc:428] Local rendezvous send item cancelled. Key hash: 11736988844813115236
2025-09-03 10:39:11.830477: W tensorflow/core/framework/op_kernel.cc:1829] UNKNOWN: InvalidArgumentError: {{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:GPU:0}} Shapes of all inputs must match: values[0].shape = [] != values[2].shape = [2] [Op:Pack] name: 1
Traceback (most recent call last):

  File "/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/p

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) UNKNOWN:  InvalidArgumentError: {{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:GPU:0}} Shapes of all inputs must match: values[0].shape = [] != values[2].shape = [2] [Op:Pack] name: 0
Traceback (most recent call last):

  File "/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/FP/cnn-for-mammograms/Ut

2025-09-03 10:39:14.879410: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: Integer division by zero
2025-09-03 10:39:14.941930: W tensorflow/core/framework/op_kernel.cc:1829] UNKNOWN: InvalidArgumentError: {{function_node __wrapped__FloorDiv_device_/job:localhost/replica:0/task:0/device:GPU:0}} Integer division by zero [Op:FloorDiv] name: 
Traceback (most recent call last):

  File "/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/FP/cnn-for-mammograms/Utils/preporcessing_utils.py", line 336, in image_prep

In [ ]:
Visualizations

## Add Layers to Basic Model

## Change Generators for Dataset Builder 

## Add Dropout 

## Search Preprocessing Technique Using Custom CNN

In [11]:
architecture_settings = {
    "iteration 1": {
        "epochs":10,
        "layers": [32, 64, 128],
        "activation": 'relu',
        "dropout": None
    },
    "iteration 1": {
        "epochs":10,
        "layers": [32, 64, 128],
        "activation": 'relu',
        "dropout": 0.2
    }
}

### Iteration 1: Basic CNN Architecture 

#### Training

In [15]:
run_model((train_data, val_data, test_data), techniques_groups)

Training Custom CNN 10 - Basic - 1Baseline Basic Preporcessing
Found 20 validated image filenames.
Found 10 validated image filenames.
Found 10 validated image filenames.
0.0 255.0 84.93707 (256, 256, 1) original
0.0 255.0 84.93707 (256, 256, 1) resize
0.0 255.0 45.41736 (256, 256, 1) original
0.0 255.0 45.41736 (256, 256, 1) resize
0.0 253.0 74.17787 (256, 256, 1) original
0.0 253.0 74.17787 (256, 256, 1) resize
0.0 255.0 58.30168 (256, 256, 1) original
0.0 255.0 58.30168 (256, 256, 1) resize
0.0 255.0 43.596542 (256, 256, 1) original
0.0 255.0 43.596542 (256, 256, 1) resize
0.0 237.0 43.18953 (256, 256, 1) original
0.0 237.0 43.18953 (256, 256, 1) resize
0.0 229.0 31.396423 (256, 256, 1) original
0.0 229.0 31.396423 (256, 256, 1) resize
0.0 255.0 55.00052 (256, 256, 1) original
0.0 255.0 55.00052 (256, 256, 1) resize
0.0 223.0 38.16356 (256, 256, 1) original
0.0 223.0 38.16356 (256, 256, 1) resize
0.0 255.0 84.50627 (256, 256, 1) original
0.0 255.0 84.50627 (256, 256, 1) resize
0.0 2

I0000 00:00:1756243517.370645  142790 service.cc:148] XLA service 0x7feac001bf90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756243517.371319  142790 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-08-26 17:25:17.456493: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1756243517.662849  142790 cuda_dnn.cc:529] Loaded cuDNN version 90501


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.5000 - loss: 0.6918

I0000 00:00:1756243521.493128  142790 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


0.0 255.0 32.133194 (256, 256, 1) original
0.0 255.0 32.133194 (256, 256, 1) resize
0.0 250.0 31.02687 (256, 256, 1) original
0.0 250.0 31.02687 (256, 256, 1) resize
0.0 255.0 55.18521 (256, 256, 1) original
0.0 255.0 55.18521 (256, 256, 1) resize
0.0 255.0 97.40271 (256, 256, 1) original
0.0 255.0 97.40271 (256, 256, 1) resize
0.0 255.0 64.18494 (256, 256, 1) original
0.0 255.0 64.18494 (256, 256, 1) resize
0.0 255.0 57.416748 (256, 256, 1) original
0.0 255.0 57.416748 (256, 256, 1) resize
0.0 253.0 37.75206 (256, 256, 1) original
0.0 253.0 37.75206 (256, 256, 1) resize
0.0 252.0 22.775742 (256, 256, 1) original
0.0 252.0 22.775742 (256, 256, 1) resize
0.0 255.0 53.002533 (256, 256, 1) original
0.0 255.0 53.002533 (256, 256, 1) resize
0.0 255.0 45.215866 (256, 256, 1) original
0.0 255.0 45.215866 (256, 256, 1) resize
0.0 255.0 32.133194 (256, 256, 1) original
0.0 255.0 32.133194 (256, 256, 1) resize
0.0 250.0 31.02687 (256, 256, 1) original
0.0 250.0 31.02687 (256, 256, 1) resize
0.0 

...

/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Found 20 validated image filenames.
Found 10 validated image filenames.
Found 10 validated image filenames.
0.0 255.0 25.989426 (256, 256, 1) original
0 255 27.752395122659312 (166, 166)
0 226 16.03828567281173 (166, 166) background i
0 226 102.30347222222223 (90, 48) crop
-4.871057656802812 210.1953846863944 102.3034689497065 (90, 48) noise
4 255 142.80069444444445 (90, 48) contrast
4 254 142.2590277777778 (90, 48) edge_enh
0.0 0.9927291 0.29637408 (256, 256, 1) resize


/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


0.0 255.0 81.07506 (256, 256, 1) original
0 255 150.4033604296705 (166, 166)
0 229 85.35436928436638 (166, 166) background i
0 229 108.15897176492228 (166, 131) crop
-3.8449260501404927 227.18748474121094 108.16365369818996 (166, 131) noise
2 255 124.28442012324106 (166, 131) contrast
2 253 123.78648946932769 (166, 131) edge_enh
0.0 0.9921569 0.3830374 (256, 256, 1) resize
0.0 255.0 58.30168 (256, 256, 1) original
0 255 133.47800841921904 (166, 166)
0 217 73.1596022644796 (166, 166) background i
0 217 93.4191844300278 (166, 130) crop
-3.9064804688353663 213.4374694824219 93.41918175443664 (166, 130) noise
2 255 113.46737720111214 (166, 130) contrast
2 253 112.98577386468953 (166, 130) edge_enh
0.0 0.9921569 0.34614906 (256, 256, 1) resize
0.0 255.0 51.162933 (256, 256, 1) original
0 255 181.2372985919582 (166, 166)
0 219 85.1155102337059 (166, 166) background i
0 219 94.19449799196786 (150, 166) crop
-4.949776208109238 212.31246948242188 94.19449477751094 (150, 166) noise
2 255 117.587

/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
# iterate trough techniques groups for training a model with each group
for technique_name, techniques in techniques_groups.items():
    
    # create model name
    model_name = "Custom CNN " + str(epochs) + " - " + "Basic - " technique_name
    print("Training " + model_name)
    
    # reset and clears variables before creating a new model 
    K.clear_session()
    
    # Create image iterators with preprocessing function for each set of preprocessing techniques 
    train_generator, val_generator, test_generator = image_iterators((train_data, val_data, test_data), 
                                                    is_resnet_vgg=False,
                                                    preprocessing_techniques=techniques
                                                  )
    
    # initiate model class
    model_instance = Basic_Custom_CNN(input_shape=(256, 256, 1), num_classes=2, epochs=epochs)
    
    # create model architecture
    model_instance.architecture()
    
    # train model
    history = model_instance.train_model(train_generator, val_gen=val_generator)
    
    # save model and get path
    name = model_name.lower().replace(" ", "_") + ".keras"
    model_path = model_instance.save_model(models_directory="Models", model_file=name)

    # evaluate model by making predictions
    evaluation = Evaluation(model_instance.get_model())
    y_probs = evaluation.predict(test_generator)

    # calculate metrics
    metrics = evaluation.calculate_metrics(y_true, y_probs)

    # get labels dictionary
    y_labels = evaluation.get_labels()

    # save data
    save_data = Save_Data(file_name="models_data.json", out_directory="Outputs")
    save_data.add_model_data(model_name, model_path, epochs, history, metrics, y_labels, project_phase, comments="")
    save_data.save_model_data()


Training Custom CNN 10 - Baseline Basic Preporcessing
Found 1889 validated image filenames.
Found 234 validated image filenames.
Found 562 validated image filenames.


/mnt/c/Users/mayra/OneDrive/Documents/MayraCSc/AI/DQNvenv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
46/60 ━━━━━━━━━━━━━━━━━━━━ 1:05 5s/step - accuracy: 0.5476 - auc: 0.5356 - loss: 0.6916 - precision: 0.5480 - recall: 0.8381

#### Visualization and Evaluation

In [ ]:
# retrieve models data 
with open('../Outputs/models_data.json', 'r') as file:
        data = json.load(file)

In [ ]:
# shows confusion matrix
vis = Visualization()